<a href="https://colab.research.google.com/github/gbrlpzz/liminal-ai-prompts/blob/main/encouraging_reuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import json
import re
from PIL import Image
import os
import io
print(f"google-generativeai version: {genai.__version__}")

google-generativeai version: 0.7.2


Load Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


API Key

In [ ]:
GOOGLE_API_KEY = userdata.get('KEY')
genai.configure(api_key=GOOGLE_API_KEY)
print(f"API Key (first 5 chars): {GOOGLE_API_KEY[:5]}...")

API Key (first 5 chars): AIzaS...


Test Connection and Load Model List

In [ ]:
try:
    models = list(genai.list_models())
    print(f"Number of models available: {len(models)}")
    for model in models:
        print(f"- {model.name}")
except Exception as e:
    print(f"Error listing models: {str(e)}")

Number of models available: 20
- models/chat-bison-001
- models/text-bison-001
- models/embedding-gecko-001
- models/gemini-1.0-pro-latest
- models/gemini-1.0-pro
- models/gemini-pro
- models/gemini-1.0-pro-001
- models/gemini-1.0-pro-vision-latest
- models/gemini-pro-vision
- models/gemini-1.5-pro-latest
- models/gemini-1.5-pro-001
- models/gemini-1.5-pro
- models/gemini-1.5-pro-exp-0801
- models/gemini-1.5-flash-latest
- models/gemini-1.5-flash-001
- models/gemini-1.5-flash
- models/gemini-1.5-flash-001-tuning
- models/embedding-001
- models/text-embedding-004
- models/aqa


Load Model

In [ ]:
try:
    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    print("Model initialized successfully")
except Exception as e:
    print(f"Error initializing model: {str(e)}")

Model initialized successfully


Analyze Image

In [ ]:
def analyze_image(image):
    try:

        model = genai.GenerativeModel('gemini-1.5-pro-latest')

        # Prepare the prompt
        prompt = """
        Analyze this image of a building and answer the following questions. For each question, choose ONLY from the provided options without providing any additional text:

        1. Person present: FALSE or TRUE
        2. Dogs/Pets present: FALSE or TRUE
        3. Livestock present: FALSE or TRUE
        4. Sounds present inside: FALSE or TRUE
        5. Lights present inside: FALSE or TRUE
        6. Collapsed Roof: FALSE or TRUE
        7. Windows: "No - Close (if all)" or "Yes - Open (if any)" or "None"
        8. Window Broken: "None" or "True (if any broken)" or "False (if all windows intact)"
        9. Shutters: "None" or "Close (if all)" or "Open (if any)"
        10. Doorway Curtains: "None" or "Unsecured" or "Secured (if any)"
        11. For Sale Sign: FALSE or TRUE
        12. Civic Number Condition: "None" or "Decorative / Tiled (if any)" or "Not Updated"
        13. Chimney Smoke: FALSE or TRUE
        14. Vehicle present in garage or private driveway: FALSE or TRUE
        15. Hanging laundry: FALSE or TRUE
        16. Personal Belongings Outside: "None" or "Good Condition" or "Mixed Condition" or "Bad Condition"
        17. Mail accumulation: "None" or "Minimal" or "Extensive"
        18. Plants / Gardens: "None" or "Well maintained (any)" or "Overgrown/Dead"
        19. Perceived Balcony Damage: "Mild Damage (cracking in plaster in facade)" or "No balcony" or "Severe Damage (visible rebar)" or "No Damage (good condition)"
        20. Perceived Private Staircase Damage: "No private staircases" or "Severe Damage (missing stairs)" or "No Damage (good condition)" or "Mild Damage (rusted railings)"
        21. Perceived Shutter Damage: "No Shutters" or "No Damage (good condition)" or "Mild Damage (needs to be repainted)" or "Severe Damage (rotting)"
        22. Door Board: "None" or "Good Condition (any)" or "Bad Condition"

        Provide your answers as a numbered list, using ONLY the exact phrases from the options given. DO NOT write any additional text, introduction or explanation.
        For examplea: 22. Door Board: "None"
        """

        # Generate content
        response = model.generate_content([prompt, image])
        clean_response = response.text
        return clean_response

    except Exception as e:
        print(f"Error analyzing image: {str(e)}")

Assign the image path and start the analysis

In [34]:
image_path = "/content/drive/MyDrive/Encouraging Reuse 2024: Fontainemore/100/17219223811500.jpeg"  # Replace with your image path
#image = compress_image(Image.open(image_path))
image = Image.open(image_path)
result = analyze_image(image)
print(result)

1. Person present: FALSE
2. Dogs/Pets present: FALSE
3. Livestock present: FALSE
4. Sounds present inside: FALSE
5. Lights present inside: FALSE
6. Collapsed Roof: FALSE
7. Windows: "No - Close (if all)"
8. Window Broken: "None"
9. Shutters: "None"
10. Doorway Curtains: "None"
11. For Sale Sign: FALSE
12. Civic Number Condition: "None"
13. Chimney Smoke: FALSE
14. Vehicle present in garage or private driveway: FALSE
15. Hanging laundry: FALSE
16. Personal Belongings Outside: "None"
17. Mail accumulation: "None"
18. Plants / Gardens: "Well maintained (any)"
19. Perceived Balcony Damage: "No balcony"
20. Perceived Private Staircase Damage: "No private staircases"
21. Perceived Shutter Damage: "No Shutters"
22. Door Board: "None"


In [35]:
def process_analysis_result(result_string, image_path):
    # Extract Building ID from image name
    building_id = re.search(r'IMG_(\d+)', image_path)
    if building_id:
        building_id = building_id.group(1)
    else:
        building_id = "Unknown"

    # Split the string into lines
    lines = result_string.split('\n')

    # Create a dictionary to store the results
    result_dict = {"Building ID": building_id}

    # Process each line
    for line in lines:
        # Split the line into key and value
        parts = line.split(': ', 1)
        if len(parts) == 2:
            key = parts[0].strip().split('. ', 1)[-1]  # Remove the number at the start
            value = parts[1].strip().strip('"')  # Remove quotes
            result_dict[key] = value

    # Convert to JSON
    json_result = json.dumps(result_dict, indent=2)
    return json_result

In [36]:
json_result = process_analysis_result(result, image_path)
print(json_result)

{
  "Building ID": "Unknown",
  "Person present": "FALSE",
  "Dogs/Pets present": "FALSE",
  "Livestock present": "FALSE",
  "Sounds present inside": "FALSE",
  "Lights present inside": "FALSE",
  "Collapsed Roof": "FALSE",
  "Windows": "No - Close (if all)",
  "Window Broken": "None",
  "Shutters": "None",
  "Doorway Curtains": "None",
  "For Sale Sign": "FALSE",
  "Civic Number Condition": "None",
  "Chimney Smoke": "FALSE",
  "Vehicle present in garage or private driveway": "FALSE",
  "Hanging laundry": "FALSE",
  "Personal Belongings Outside": "None",
  "Mail accumulation": "None",
  "Plants / Gardens": "Well maintained (any)",
  "Perceived Balcony Damage": "No balcony",
  "Perceived Private Staircase Damage": "No private staircases",
  "Perceived Shutter Damage": "No Shutters",
  "Door Board": "None"
}


In [37]:
def calculate_occupancy_rating(result_dict):
    point_system = {
        "Person present": {"FALSE": 0, "TRUE": 4},
        "Dogs/Pets present": {"FALSE": 0, "TRUE": 3},
        "Livestock present": {"FALSE": 0, "TRUE": 2},
        "Sounds present inside": {"FALSE": 0, "TRUE": 2},
        "Lights present inside": {"FALSE": 0, "TRUE": 3},
        "Collapsed Roof": {"FALSE": 0, "TRUE": -6},
        "Windows": {"No - Close (if all)": 0, "Yes - Open (if any)": 2, "None": -3},
        "Window Broken": {"None": 0, "True (if any broken)": -2, "False (if all windows intact)": 1},
        "Shutters": {"None": 0, "Close (if all)": -1, "Open (if any)": 2},
        "Doorway Curtains": {"None": 0, "Unsecured": 2, "Secured (if any)": 2},
        "For Sale Sign": {"FALSE": 0, "TRUE": -2},
        "Civic Number Condition": {"None": 0, "Decorative / Tiled (if any)": 1, "Not Updated": 0},
        "Chimney Smoke": {"FALSE": 0, "TRUE": 3},
        "Vehicle present in garage or private driveway": {"FALSE": 0, "TRUE": 2},
        "Hanging laundry": {"FALSE": 0, "TRUE": 4},
        "Personal Belongings Outside": {"None": -1, "Good Condition": 3, "Mixed Condition": 2, "Bad Condition": -1},
        "Mail accumulation": {"None": 0, "Minimal": -1, "Extensive": -1},
        "Plants / Gardens": {"None": 0, "Well maintained (any)": 1, "Overgrown/Dead": -2},
        "Perceived Balcony Damage": {"Mild Damage (cracking in plaster in facade)": 0, "No balcony": 0, "Severe Damage (visible rebar)": -3, "No Damage (good condition)": 2},
        "Perceived Private Staircase Damage": {"No private staircases": 0, "Severe Damage (missing stairs)": -2, "No Damage (good condition)": 1, "Mild Damage (rusted railings)": 0},
        "Perceived Shutter Damage": {"No Shutters": 0, "No Damage (good condition)": 1, "Mild Damage (needs to be repainted)": 0, "Severe Damage (rotting)": -2},
        "Door Board": {"None": 0, "Good Condition (any)": 1, "Bad Condition": -2}
    }

    total_points = 0
    for key, value in result_dict.items():
        if key in point_system:
            total_points += point_system[key].get(value, 0)

    return total_points

def process_json_result(json_string):
    # Parse the JSON string
    result_dict = json.loads(json_string)

    # Calculate occupancy rating
    occupancy_rating = calculate_occupancy_rating(result_dict)
    result_dict["Occupancy Rating"] = occupancy_rating

    # Convert back to JSON
    updated_json_result = json.dumps(result_dict, indent=2)
    return updated_json_result

updated_result = process_json_result(json_result)
print(updated_result)

{
  "Building ID": "Unknown",
  "Person present": "FALSE",
  "Dogs/Pets present": "FALSE",
  "Livestock present": "FALSE",
  "Sounds present inside": "FALSE",
  "Lights present inside": "FALSE",
  "Collapsed Roof": "FALSE",
  "Windows": "No - Close (if all)",
  "Window Broken": "None",
  "Shutters": "None",
  "Doorway Curtains": "None",
  "For Sale Sign": "FALSE",
  "Civic Number Condition": "None",
  "Chimney Smoke": "FALSE",
  "Vehicle present in garage or private driveway": "FALSE",
  "Hanging laundry": "FALSE",
  "Personal Belongings Outside": "None",
  "Mail accumulation": "None",
  "Plants / Gardens": "Well maintained (any)",
  "Perceived Balcony Damage": "No balcony",
  "Perceived Private Staircase Damage": "No private staircases",
  "Perceived Shutter Damage": "No Shutters",
  "Door Board": "None",
  "Occupancy Rating": 0
}
